In [2]:
import pytesseract
from PIL import Image
import os

# Path to the Tesseract executable (change this according to your setup)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Users\USER\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
ocr_tesseract_path = os.path.join(os.getcwd(), "tesseract_ocr/tesseract.exe")
pytesseract.pytesseract.tesseract_cmd = ocr_tesseract_path

def parse_receipt(image_path):
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
        return text

# Path to your receipt image
image_path = os.path.join(os.getcwd(), "sample_receipts/walmart01.png")
receipt_text = parse_receipt(image_path)
print(receipt_text)


PermissionError: [Errno 13] Permission denied: '/Users/aarnavgogri/Desktop/receipt-plus/source/backend/tesseract_ocr/tesseract.exe'